# Import libraries

In [23]:
import sklearn
import ast
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import plotly.express as px
from scipy.stats import randint
from scipy.stats import uniform
import tensorflow as tf

#Linear model
from sklearn.linear_model import LinearRegression

#KNN
from sklearn.neighbors import KNeighborsRegressor

#RF
from sklearn.ensemble import RandomForestRegressor

#SVR
from sklearn.svm import SVR

#Metrics
from sklearn.model_selection import cross_validate, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error

#Feature selection
from sklearn.feature_selection import RFE
from sklearn.svm import SVR

#Lasso
from sklearn.linear_model import Lasso

#Ridge
from sklearn.linear_model import Ridge

#Gradient Boosting 
#lightgbm
import lightgbm as lgb
#catboost
import catboost as ctb
#xgboost
import xgboost as xgb
import matplotlib as mpl

#Neural network
import math
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler


# Download data

In [24]:
data = pd.read_csv('final_data_without_prefs_and_with_HAMADA_factors.csv')
data_ret = pd.read_csv('data_ret.csv')
data_ret.drop(columns=['Unnamed: 0'], inplace=True)
#data_ret.set_index('Date', inplace=True)
data.set_index('Date', inplace = True)
macro_data = pd.read_csv('macro_data.csv')
macro_data.set_index('Date', inplace = True)

# Tune ML models and their hyperparameters

In [ ]:
tf.random.set_seed(42)

ml_results = pd.DataFrame({'share':[1],
                           'SVM_R2_train':[1], 'SVM_R2_test':[1],
                           'RF_R2_train':[1], 'RF_R2_test':[1],
                           'LinReg_R2_train':[1], 'LinReg_R2_test':[1],
                           'KNN_R2_train':[1], 'KNN_R2_test':[1],
                           'LASSO_R2_train':[1], 'LASSO_R2_test':[1],
                           'Ridge_R2_train':[1], 'Ridge_R2_test':[1],
                           'Light_GBM_R2_train':[1], 'Light_GBM_R2_test':[1],
                           'XGBoost_R2_train':[1], 'XGBoost_R2_test':[1],
                           'Catboost_R2_train':[1], 'Catboost_R2_test':[1],
                           'NN_R2_train':[1], 'NN_R2_test':[1]})

ml_results_1 = pd.DataFrame({'share':[1],
  'SVM_MSE_train':[1], 'SVM_MSE_test':[1],
  'RF_MSE_train':[1], 'RF_MSE_test':[1],
  'LinReg_MSE_train':[1], 'LinReg_MSE_test':[1],
  'KNN_MSE_train':[1], 'KNN_MSE_test':[1],
  'LASSO_MSE_train':[1], 'LASSO_MSE_test':[1],
  'Ridge_MSE_train':[1], 'Ridge_MSE_test':[1],
  'Light_GBM_MSE_train':[1], 'Light_GBM_MSE_test':[1],
  'XGBoost_MSE_train':[1], 'XGBoost_MSE_test':[1],
  'Catboost_MSE_train':[1], 'Catboost_MSE_test':[1],
  'NN_MSE_train':[1], 'NN_MSE_test':[1]})

ml_results_best_params = pd.DataFrame({'share':[1],
                                      'SVM_best_params':[1],
                                      'RF_best_params':[1], 
                                      'LinReg_best_params':[1],
                                      'KNN_best_params':[1],
                                      'LASSO_best_params':[1],
                                      'Ridge_best_params':[1],
                                      'Light_GBM_best_params':[1], 
                                      'XGBoost_best_params':[1],
                                      'Catboost_best_params':[1],
                                      'NN_best_params':[1]})


lag_autolag_return_30 = 30
lag_autolag_return_60 = 60
lag_autolag_return_90 = 90
lag_30 = 30
lag_60 = 60
lag_90 = 90


for name in data_ret.columns[2:]:
    
    df1 = data[[name, 'Hamada_beta_MRP_for_{}'.format(name[7:11]), 'risk_free_rate_based_on_g_curve',
                 'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
                 'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]),
                 'MOM']].dropna()
    df_autolag = pd.DataFrame(data[name] - data['risk_free_rate_based_on_g_curve']).rename(columns = {0:'{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_30)})
    df_autolag = pd.DataFrame(df_autolag['{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_30)].shift(lag_autolag_return_30).dropna())
    
    df_autolag_2 = pd.DataFrame(data[name] - data['risk_free_rate_based_on_g_curve']).rename(columns = {0:'{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_60)})
    df_autolag_2 = pd.DataFrame(df_autolag_2['{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_60)].shift(lag_autolag_return_60).dropna())
    
    df_autolag_3 = pd.DataFrame(data[name] - data['risk_free_rate_based_on_g_curve']).rename(columns = {0:'{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_90)})
    df_autolag_3 = pd.DataFrame(df_autolag_3['{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_90)].shift(lag_autolag_return_90).dropna())
    
    df2 = data[['Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
            'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 'MOM']].shift(lag_30).dropna()
    
    
    
    df2.rename(columns = {'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_30),
                          'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_30), 
                          'MOM':'MOM_lag_{}_days'.format(lag_30)}, inplace = True)
    
    result = pd.concat([df1, df2], axis=1, join="inner")
    
    df3 = data[['Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
            'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 'MOM']].shift(lag_60).dropna()
    
    df3.rename(columns = {'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_60),
                          'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_60), 
                          'MOM':'MOM_lag_{}_days'.format(lag_60)}, inplace = True)
    
    result = pd.concat([result, df3], axis=1, join="inner")
    
    df4 = data[['Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
            'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 'MOM']].shift(lag_90).dropna()
    
    df4.rename(columns = {'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_90),
                          'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_90), 
                          'MOM':'MOM_lag_{}_days'.format(lag_90)}, inplace = True)
    
    result = pd.concat([result, df4], axis=1, join="inner")
    
    result = pd.merge(result, macro_data, left_index=True, right_index=True)
    result = pd.merge(result, df_autolag, left_index=True, right_index=True)
    result = pd.merge(result, df_autolag_2, left_index=True, right_index=True)
    result = pd.merge(result, df_autolag_3, left_index=True, right_index=True)
    result['{}_Equity_premium'.format(name[7:14])] = result[name] - result['risk_free_rate_based_on_g_curve']
      
    share = '{}_Equity_premium'.format(name[7:14])

    print("###############################################")
    
    R_2 = []
    R_2.append(share)
    
    MSE = []
    MSE.append(share)
    
    best_params = []
    best_params.append(share)
    
    print("ML models for {}".format(share))
    dta = result.dropna()
    
    X_train = dta.iloc[:round(len(dta) - len(dta)/4),:][['Hamada_beta_MRP_for_{}'.format(name[7:11]),
       '{}_premium_lag_{}_days'.format(name[7:], lag_30),
       'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
       'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 
       'MOM',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_30),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_30), 
       'MOM_lag_30_days',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_60),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_60), 
       'MOM_lag_60_days',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_90),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_90),  
       'MOM_lag_90_days',
       'dollar_30d_return', 'wti_30d_return']]
    X_test = dta.iloc[-round(len(dta)/4):,:][['Hamada_beta_MRP_for_{}'.format(name[7:11]),
       '{}_premium_lag_{}_days'.format(name[7:], lag_30),
       'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
       'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 
       'MOM',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_30),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_30), 
       'MOM_lag_30_days',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_60),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_60), 
       'MOM_lag_60_days',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_90),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_90),  
       'MOM_lag_90_days',
       'dollar_30d_return', 'wti_30d_return']]
    y_train = dta.iloc[:round(len(dta) - len(dta)/4),:][share]
    y_test = dta.iloc[-round(len(dta)/4):,:][share]

    print("SVM")
    svr = SVR()
    distributions = {'C': loguniform(1e0, 1e3),
                     'gamma': loguniform(1e-4, 1e-3),
                     'kernel': ['rbf', 'sigmoid', 'linear', 'poly']}

    svr = RandomizedSearchCV(svr, distributions, random_state=0, n_jobs=-1)
    svr = svr.fit(X_train, y_train)
    y_pred_train = svr.best_estimator_.predict(X_train)
    print(svr.best_params_)
    best_params.append(svr.best_params_)
    print('R2 : {}'.format(r2_score(y_train, y_pred_train)))
    print('MSE : {}'.format(mean_squared_error(y_train, y_pred_train)))

    R_2.append(r2_score(y_train, y_pred_train))
    MSE.append(mean_squared_error(y_train, y_pred_train))

    y_pred = svr.predict(X_test)
    print('R2 : {}'.format(r2_score(y_test, y_pred)))
    print('MSE : {}'.format(mean_squared_error(y_test, y_pred)))


    R_2.append(r2_score(y_test, y_pred))
    MSE.append(mean_squared_error(y_test, y_pred))
    print("#########")


    print("RandomForest")
    rf = RandomForestRegressor(n_estimators=100,criterion='mse',max_depth=10,oob_score=True, bootstrap = True,
                                  n_jobs = -1, verbose = 1, random_state = 42)

    distributions = {'bootstrap': [True, False],
                     'max_depth': [3,4,5,6,7,9,10, 15, None],
                     'max_features': ['auto', 'sqrt'],
                     'min_samples_leaf': [1, 2, 4, 10, 30, 50, 70],
                     'min_samples_split': [2,3, 5,7, 10],
                     'n_estimators': [20,50, 100, 150, 200, 400]}

    rf = RandomizedSearchCV(rf, distributions, random_state=0, n_iter = 40, cv = 3, verbose=2, n_jobs = -1)
    rf = rf.fit(X_train, y_train)
    y_pred = rf.best_estimator_.predict(X_train)
    print(rf.best_params_)
    best_params.append(rf.best_params_)
    print('train R2 : {}'.format(r2_score(y_train, y_pred)))
    print('train MSE : {}'.format(mean_squared_error(y_train, y_pred)))

    R_2.append(r2_score(y_train, y_pred))
    MSE.append(mean_squared_error(y_train, y_pred))

    y_pred = rf.predict(X_test)
    print('test R2 : {}'.format(r2_score(y_test, y_pred)))
    print('test MSE : {}'.format(mean_squared_error(y_test, y_pred)))
    R_2.append(r2_score(y_test, y_pred))
    MSE.append(mean_squared_error(y_test, y_pred))

    print("#########")

    print("LinearRegression")
    model = LinearRegression(fit_intercept=False)

    model.fit(X_train,y_train)
    y_pred = model.predict(X_train)
    print('train R2 : {}'.format(r2_score(y_train, y_pred)))
    print('train MSE : {}'.format(mean_squared_error(y_train, y_pred)))

    R_2.append(r2_score(y_train, y_pred))
    MSE.append(mean_squared_error(y_train, y_pred))

    y_pred = model.predict(X_test)
    print('test R2 : {}'.format(r2_score(y_test, y_pred)))
    print('test MSE : {}'.format(mean_squared_error(y_test, y_pred)))
    R_2.append(r2_score(y_test, y_pred))
    MSE.append(mean_squared_error(y_test, y_pred))
    print("#########")

    print("KNN")
    knn = KNeighborsRegressor()

    distributions = {'n_neighbors': [3,5,7,10,15,25,40,60, 80,100, 150],
                     'weights': ['uniform', 'distance'],
                     'algorithm':['auto', 'ball_tree', 'kd_tree', 'brute'],
                     'p':[1,2]}

    knn = RandomizedSearchCV(knn, distributions, random_state=0, n_iter = 40, cv = 3, verbose=2, n_jobs = -1)
    knn = knn.fit(X_train, y_train)
    y_pred = knn.best_estimator_.predict(X_train)

    knn.fit(X=X_train, y=y_train)
    y_pred = knn.predict(X_train)

    print(knn.best_params_)
    best_params.append(knn.best_params_)
    print('train R2 : {}'.format(r2_score(y_train, y_pred)))
    print('train MSE : {}'.format(mean_squared_error(y_train, y_pred)))

    R_2.append(r2_score(y_train, y_pred))
    MSE.append(mean_squared_error(y_train, y_pred))
    y_pred = knn.predict(X_test)

    print('test R2 : {}'.format(r2_score(y_test, y_pred)))
    print('test MSE : {}'.format(mean_squared_error(y_test, y_pred)))
    R_2.append(r2_score(y_test, y_pred))
    MSE.append(mean_squared_error(y_test, y_pred))
    print("#########")

    print("LASSO")
    lasso = Lasso()
    distributions = {'alpha':[0.000001, 0.0001, 0.001, 0.01, 0.1, 1],
                     'tol':[0.000001, 0.0001, 0.001, 0.01, 0.1, 1, 10],
                     'normalize':[True, False],
                     'random_state':[0]}

    lasso = RandomizedSearchCV(lasso, distributions, random_state=0, n_iter = 40, cv = 3, verbose=2, n_jobs = -1)
    lasso = lasso.fit(X_train, y_train)
    y_pred = lasso.best_estimator_.predict(X_train)

    lasso.fit(X=X_train, y=y_train)
    y_pred = lasso.predict(X_train)

    print(lasso.best_params_)
    best_params.append(lasso.best_params_)
    print('train R2 : {}'.format(r2_score(y_train, y_pred)))
    print('train MSE : {}'.format(mean_squared_error(y_train, y_pred)))
    R_2.append(r2_score(y_train, y_pred))
    MSE.append(mean_squared_error(y_train, y_pred))
    y_pred = lasso.predict(X_test)

    print('test R2 : {}'.format(r2_score(y_test, y_pred)))
    print('test MSE : {}'.format(mean_squared_error(y_test, y_pred)))
    R_2.append(r2_score(y_test, y_pred))
    MSE.append(mean_squared_error(y_test, y_pred))

    print("#########")

    print("Ridge")
    ridge = Ridge()

    distributions = {'alpha':[0.000001, 0.0001, 0.001, 0.01, 0.1, 1],
                     'tol':[0.000001, 0.0001, 0.001, 0.01, 0.1, 1, 10],
                     'normalize':[True, False],
                     'random_state':[0]}

    ridge = RandomizedSearchCV(ridge, distributions, random_state=0, n_iter = 40, cv = 3, verbose=2, n_jobs = -1)
    ridge = ridge.fit(X_train, y_train)
    y_pred = ridge.best_estimator_.predict(X_train)

    print(ridge.best_params_)
    best_params.append(ridge.best_params_)
    print('train R2 : {}'.format(r2_score(y_train, y_pred)))
    print('train MSE : {}'.format(mean_squared_error(y_train, y_pred)))
    R_2.append(r2_score(y_train, y_pred))
    MSE.append(mean_squared_error(y_train, y_pred))
    y_pred = ridge.predict(X_test)

    print('test R2 : {}'.format(r2_score(y_test, y_pred)))
    print('test MSE : {}'.format(mean_squared_error(y_test, y_pred)))
    R_2.append(r2_score(y_test, y_pred))
    MSE.append(mean_squared_error(y_test, y_pred))

    print("#########")

    print("Light GBM")
    model = lgb.LGBMRegressor(random_state=0)

    distributions = {'learning_rate':[0.000001, 0.0001, 0.001, 0.01, 0.05, 0.1, 0.5, 1],
                     'max_depth' :[1,2,3,4,5,-1],
                     'n_estimators':[100,200, 300, 400, 600],
                     'boosting_type':['gbdt', 'dart', 'goss', 'rf'],
                     'random_state':[0],
                     'min_child_samples':[3,7, 10,20,40, 60],
                     'importance_type':['split', 'gain'],
                     'num_leaves': randint(6, 50), 
                     'min_child_samples': randint(100, 500), 
                     'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
                     'subsample': uniform(loc=0.2, scale=0.8), 
                     'colsample_bytree': uniform(loc=0.4, scale=0.6),
                     'reg_alpha': [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
                     'reg_lambda': [0, 1e-1, 1, 5, 10, 20, 50, 100]}

    model = RandomizedSearchCV(model, distributions, random_state=0, n_iter = 100, cv = 8, verbose=2, n_jobs = -1)
    model = model.fit(X_train, y_train)
    y_pred = model.best_estimator_.predict(X_train)

    print(model.best_params_)
    best_params.append(model.best_params_)
    model.fit(X=X_train, y=y_train)
    y_pred = model.predict(X=X_train)

    print('train R2 : {}'.format(r2_score(y_train, y_pred)))
    print('train MSE : {}'.format(mean_squared_error(y_train, y_pred)))
    R_2.append(r2_score(y_train, y_pred))
    MSE.append(mean_squared_error(y_train, y_pred))
    y_pred = model.predict(X=X_test)

    print('test R2 : {}'.format(r2_score(y_test, y_pred)))
    print('test MSE : {}'.format(mean_squared_error(y_test, y_pred)))
    R_2.append(r2_score(y_test, y_pred))
    MSE.append(mean_squared_error(y_test, y_pred))

    print("#########")

    print("XGBoost")
    model = xgb.XGBRegressor(random_state=0)

    distributions = {'learning_rate':[0.000001, 0.0001, 0.001, 0.01,0.05, 0.1, 0.5, 1],
                     'max_depth' :[1,2,3,4,5,-1],
                     'n_estimators':[100,200, 300, 400, 600],
                     'random_state':[0],
                     'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
                     'subsample': uniform(loc=0.2, scale=0.8), 
                     'colsample_bytree': uniform(loc=0.4, scale=0.6),
                     'reg_alpha': [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
                     'reg_lambda': [0, 1e-1, 1, 5, 10, 20, 50, 100]}

    model = RandomizedSearchCV(model, distributions, random_state=0, n_iter = 100, cv = 8, verbose=2, n_jobs = -1)
    model = model.fit(X_train, y_train)
    y_pred = model.best_estimator_.predict(X_train)

    print(model.best_params_)
    best_params.append(model.best_params_)
    print('train R2 : {}'.format(r2_score(y_train, y_pred)))
    print('train MSE : {}'.format(mean_squared_error(y_train, y_pred)))
    R_2.append(r2_score(y_train, y_pred))
    MSE.append(mean_squared_error(y_train, y_pred))
    y_pred = model.predict(X_test)

    print('test R2 : {}'.format(r2_score(y_test, y_pred)))
    print('test MSE : {}'.format(mean_squared_error(y_test, y_pred)))
    R_2.append(r2_score(y_test, y_pred))
    MSE.append(mean_squared_error(y_test, y_pred))
    print("#########")

    print("Catboost")
    model = ctb.CatBoostRegressor(random_state=0)



    distributions = {'learning_rate':[0.000001, 0.0001, 0.001, 0.01,0.05, 0.1, 0.5, 1],
                     'depth' :[1,2,3,4,5],
                     'iterations':randint(10,100),
                     'l2_leaf_reg': [1, 3, 5, 7, 9], 'silent':[True]}

    model = RandomizedSearchCV(model, distributions, random_state=0, n_iter = 20, cv = 3)
    model = model.fit(X_train, y_train)
    y_pred = model.best_estimator_.predict(X_train)

    print(model.best_params_)
    best_params.append(model.best_params_)
    print('train R2 : {}'.format(r2_score(y_train, y_pred)))
    print('train MSE : {}'.format(mean_squared_error(y_train, y_pred)))
    R_2.append(r2_score(y_train, y_pred))
    MSE.append(mean_squared_error(y_train, y_pred))
    y_pred = model.predict(X_test)

    print('test R2 : {}'.format(r2_score(y_test, y_pred)))
    print('test MSE : {}'.format(mean_squared_error(y_test, y_pred)))
    R_2.append(r2_score(y_test, y_pred))
    MSE.append(mean_squared_error(y_test, y_pred))
    print("#########")
    print("Multilayer Perceptron")

    look_back = X_train.shape[1]
    model = Sequential()
    model.add(Dense(4, input_dim=look_back, activation='tanh'))
    model.add(Dense(8, input_dim=look_back, activation='tanh'))
    model.add(Dense(32, input_dim=look_back, activation='tanh'))
    model.add(Dense(64, input_dim=look_back, activation='tanh'))
    model.add(Dense(32, input_dim=look_back, activation='tanh'))
    model.add(Dense(16, input_dim=look_back, activation='tanh'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.fit(X_train, y_train, epochs=7, batch_size=2, verbose=0)


    y_pred = model.predict(X_train)

    print('train R2 : {}'.format(r2_score(y_train, y_pred)))
    print('train MSE : {}'.format(mean_squared_error(y_train, y_pred)))
    R_2.append(r2_score(y_train, y_pred))
    MSE.append(mean_squared_error(y_train, y_pred))
    y_pred = model.predict(X_test)

    print('test R2 : {}'.format(r2_score(y_test, y_pred)))
    print('test MSE : {}'.format(mean_squared_error(y_test, y_pred)))
    R_2.append(r2_score(y_test, y_pred))
    MSE.append(mean_squared_error(y_test, y_pred))
    print("#########")

    df = pd.DataFrame(R_2).T
    df_1 = pd.DataFrame(MSE).T
    df_2 = pd.DataFrame(best_params).T
    
    df.rename(columns={0:'share',
                       1:'SVM_R2_train', 2:'SVM_R2_test',
                       3:'RF_R2_train', 4:'RF_R2_test',
                       5:'LinReg_R2_train', 6:'LinReg_R2_test',
                       7:'KNN_R2_train', 8:'KNN_R2_test',
                       9:'LASSO_R2_train', 10:'LASSO_R2_test',
                       11:'Ridge_R2_train', 12:'Ridge_R2_test',
                       13:'Light_GBM_R2_train',14:'Light_GBM_R2_test',
                       15:'XGBoost_R2_train', 16:'XGBoost_R2_test',
                       17:'Catboost_R2_train', 18:'Catboost_R2_test',
                       19:'NN_R2_train', 20:'NN_R2_test'},inplace=True)

    df_1.rename(columns={0:'share',
                       1:'SVM_MSE_train', 2:'SVM_MSE_test',
                       3:'RF_MSE_train', 4:'RF_MSE_test',
                       5:'LinReg_MSE_train', 6:'LinReg_MSE_test',
                       7:'KNN_MSE_train', 8:'KNN_MSE_test',
                       9:'LASSO_MSE_train', 10:'LASSO_MSE_test',
                       11:'Ridge_MSE_train', 12:'Ridge_MSE_test',
                       13:'Light_GBM_MSE_train',14:'Light_GBM_MSE_test',
                       15:'XGBoost_MSE_train', 16:'XGBoost_MSE_test',
                       17:'Catboost_MSE_train', 18:'Catboost_MSE_test',
                       19:'NN_MSE_train', 20:'NN_MSE_test'},inplace=True)
    
    df_2.rename(columns={0:'share',
                       1:'SVM_best_params', 2:'RF_best_params',
                       3:'KNN_best_params', 4:'LASSO_best_params',
                       5:'Ridge_best_params', 6:'Light_GBM_best_params',
                       7:'XGBoost_best_params', 8:'Catboost_best_params'},inplace=True)
    
    
    ml_results = pd.concat([ml_results, df])
    ml_results_1 = pd.concat([ml_results_1, df_1])
    ml_results_best_params = pd.concat([ml_results_best_params, df_2])
    

ml_results = ml_results.iloc[1:,:]
ml_results_1 = ml_results_1.iloc[1:,:]
ml_results_best_params = ml_results_best_params.iloc[1:,:]

ml_data = ml_results.set_index('share')
ml_data_1 = ml_results_1.set_index('share')
ml_results_best_params = ml_results_best_params.set_index('share')

# ml_results_best_params.to_csv('ML_best_params_ever.csv')
# ml_data_1.to_csv('MSE_result_ever.csv')
# ml_data.to_csv('R2_result_ever.csv')

# Read received results

In [25]:
ml_results_best_params = pd.read_csv('ML_best_params_ever.csv')
ml_data_1 = pd.read_csv('MSE_result_ever.csv')
ml_data = pd.read_csv('R2_result_ever.csv')

ml_data.set_index('share', inplace=True)
ml_data_1.set_index('share', inplace=True)
ml_results_best_params.set_index('share', inplace=True)

# Plot the results

## R2

In [26]:
my_df_alg = ml_data.filter(regex = 'test').T
my_df_alg['test_R2'] = ml_data.filter(regex = 'test').T.mean(axis = 1)
my_df_alg['train_R2'] = ml_data.filter(regex = 'train').T.mean(axis = 1)
my_df = pd.DataFrame(ml_data.mean(axis=0)).rename(columns={0:'R2'})

In [27]:
fig = px.line(my_df.T.filter(regex = 'train').T, y = 'R2')
fig.update_layout(xaxis_title = 'Алгоритм')
fig.show()

In [28]:
fig = px.line(my_df.T.filter(regex = 'test').T, y = 'R2')
fig.update_layout(xaxis_title = 'Алгоритм')
fig.show()

## MSE

In [29]:
ml_data['Средний MSE на тестовой выборке'] = ml_data_1.filter(regex = 'test').mean(axis = 1)
ml_data['Средний MSE на обучении'] = ml_data_1.filter(regex = 'train').mean(axis = 1)
ml_data['Акция'] = ml_data_1.index.astype(str).str[:4]

In [30]:
names = {'SBER':'Сбербанк', 'GAZP':'Газпром', 'LKOH':'Лукойл','GMKN':'Норникель' ,'NVTK':'Новатэк', 'SNGS':'Сургутнефтегаз',
         'PLZL':'Полюс','TATN':'Татнефть','ROSN':'Роснефть','MGNT':'Магнит','MTSS':'МТС', 'MOEX':'МосБиржа', 
         'IRAO':'Интер РАО', 'NLMK':'НЛМК', 'ALRS':'Алроса','CHMF':'Северсталь','VTBR':'ВТБ','RTKM':'Ростелеком',
         'PHOR':'Фосагро','AFKS':'АФК «Система»','MAGN':'ММК','PIKK':'ПИК',
         'HYDR':'Русгидро','FEES':'ФСК ЕЭС','AFLT':'Аэрофлот','LSRG':'ЛСР','RSTI':'Россети','UPRO':'Юнипро',
         'LNTA':'Лента','MVID':'М.Видео','TRMK':'ТМК','MTLR':'Мечел','NMTP':'НМТП','AKRN':'Акрон',
         'MSNG':'Мосэнерго','URKA':'Уралкалий','BANE':'Башнефть','VSMO':'ВСМПО-Ависма','NKNC':'Нижнекамскнефтехим',
         'GCHE':'Черкизово','SVAV':'Соллерс','MSTT':'Мостотрест','BSPB':'Банк Санкт-Петербург', 'MSRS':'МОЭСК',
         'KMAZ':'Камаз','RASP':'Распадская','OGKB':'ОГК №2','VZRZ':'Банк "Возрождение"'}
ml_data.replace({"Акция": names}, inplace=True)

In [31]:
import plotly.graph_objects as go
fig = go.Figure()


fig.add_trace(go.Scatter(x=ml_data['Акция'], y=ml_data['Средний MSE на тестовой выборке'],
                         mode='lines+markers',name='Средний MSE на тестовой выборке'))
fig.add_trace(go.Scatter(x=ml_data['Акция'], y=ml_data['Средний MSE на обучении'],
                         mode='lines+markers',name='Средний MSE на обучающей выборке'))
fig.update_layout(yaxis_title = 'MSE',xaxis_title = 'Акция',autosize=False,width=1300,height = 700,
                  legend=dict(yanchor="top", y=0.96, xanchor="left", x=0.01))

fig.show()

In [32]:
my_df_alg = ml_data_1.filter(regex = 'test').T
my_df_alg['test_MSE'] = ml_data_1.filter(regex = 'test').T.mean(axis = 1)
my_df_alg['train_MSE'] = ml_data_1.filter(regex = 'train').T.mean(axis = 1)
my_df = pd.DataFrame(ml_data_1.mean(axis=0)).rename(columns={0:'MSE'})


fig = go.Figure()
fig.add_trace(go.Scatter(x=my_df.T.filter(regex = 'train').columns.astype(str).str[:-10], y=my_df.T.filter(regex = 'train').T['MSE'],
                         mode='lines+markers',name='Средний MSE на обучающей выборке'))
fig.add_trace(go.Scatter(x=my_df.T.filter(regex = 'test').columns.astype(str).str[:-9], y=my_df.T.filter(regex = 'test').T['MSE'],
                         mode='lines+markers',name='Средний MSE на тестовой выборке'))

fig.update_layout(yaxis_title = 'MSE',xaxis_title = 'Алгоритм',autosize=False,width=1300,
                  legend=dict(yanchor="top", y=0.55, xanchor="left", x=0.1))
fig.show()